# Metody pozyskiwania wiedzy z danych - projekt zaliczeniowy

Narzędzie pozwalające użytkownikowi na estymację ceny mieszkania w zależności od podanych parametrów.

Skład zespołu: 
- Zapalski, 
- Szkatuła, 
- Pietrzyk, 
- Śmiałowski

## To do:
- opis zestawu danych i jego źródła
- analiza eksploracyjna zestawu danych (info o zmiennych, statystki opisowe, rozkłady, korelacje, .... )
- analiza wizualna (przedstawienie danych w formie wizualnej, przeprowadzenie analiz zależności pomiędzy zmiennymi, heatmapy z najczęstszymi miejscami, ... )
- ocztszczenie danych (outliery, NA itd. )
- pierwszy model
- feature enginering
- zastosowanie różnych typów modeli (?)
- hyperparameters tuning na najlepszym modelu
- stworzenie coś na wzór UX w noteboooku

## Notes:
- zrobić coś na wzór UI dla użytkownika końcowego (może być jako suwaczki w jupyterze albo API)
- lista dzielnic i miast, feature enginering na zmiennej adres tak zeby wyciagnac dzielnice
- analiza danych, statystki opisowe, wykresy i mapki

## Code

In [108]:
import pandas as pd
import matplotlib as plt
import numpy as np

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeRegressor
import catboost as ctb

In [109]:
!pwd

/Users/mikolaj/Desktop/2 semestr/wiedza-dane/!projekt


In [110]:
df = pd.read_csv("data/Houses.csv", encoding='latin-1',index_col=0)
df.head()

,address,city,floor,id,latitude,longitude,price,rooms,sq,year
0,Podgórze Zab³ocie Stanis³awa Klimeckiego,Kraków,2.0,23918.0,50.049224,19.970379,749000.0,3.0,74.05,2021.0
1,Praga-Po³udnie Grochowska,Warszawa,3.0,17828.0,52.249775,21.106886,240548.0,1.0,24.38,2021.0
2,Krowodrza Czarnowiejska,Kraków,2.0,22784.0,50.066964,19.920025,427000.0,2.0,37.00,1970.0
3,Grunwald,Poznañ,2.0,4315.0,52.404212,16.882542,1290000.0,5.0,166.00,1935.0
4,Ochota Gotowy budynek. Stan deweloperski. Osta...,Warszawa,1.0,11770.0,52.212225,20.972630,996000.0,5.0,105.00,2020.0


In [111]:
# wybranie zmiennych jakościowych
obj_feats = list(df.select_dtypes(np.object).columns)

# zamiana ich na liczby
for feat in obj_feats:
    df["{}_cat".format(feat)] = df[feat].factorize()[0]

black_list = ["address", "city"]

feats = df.select_dtypes(include=[np.number, np.bool]).columns
feats = [x for x in feats if x not in black_list]

X = df[feats].to_numpy()
y = df["price"].to_numpy()

# podział na train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [112]:
model = ctb.CatBoostRegressor(verbose=False)
#cross_val_score(model, X_train, y_train, cv=3, scoring='r2')

In [113]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [114]:
r2_score(y_test, y_pred)

0.9318582441888341

In [117]:
df["price"].median()

520000.0

In [116]:
rms = mean_squared_error(y_test, y_pred, squared=False)
rms

147359.68873493944